# Задание 1

Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

In [5]:
import pandas as pd

import datetime
from operator import attrgetter

In [6]:
auth_data = pd.read_csv('problem1-auth_data.csv', sep=';')
auth_data.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [7]:
auth_data.shape

(9601013, 2)

In [8]:
auth_data.nunique()

auth_ts    9180915
uid        1000000
dtype: int64

In [9]:
auth_data.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [10]:
auth_data.duplicated().sum()

0

In [11]:
reg_data = pd.read_csv('problem1-reg_data.csv', sep=';')
reg_data.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [12]:
reg_data.shape

(1000000, 2)

In [13]:
reg_data.nunique()

reg_ts    1000000
uid       1000000
dtype: int64

In [14]:
reg_data.isna().sum()

reg_ts    0
uid       0
dtype: int64

In [15]:
reg_data.duplicated().sum()

0

In [16]:
df = auth_data.merge(reg_data, how='left', on='uid') # объединяем данные
df['reg_date'] = pd.to_datetime(df.reg_ts, unit='s').dt.to_period('D') # преобразуем данные в дни
df['auth_date'] = pd.to_datetime(df.auth_ts, unit='s').dt.to_period('D') # преобразуем данные в дни

In [17]:
df.head()

,auth_ts,uid,reg_ts,reg_date,auth_date
0,911382223,1,911382223,1998-11-18,1998-11-18
1,932683089,2,932683089,1999-07-22,1999-07-22
2,932921206,2,932683089,1999-07-22,1999-07-25
3,933393015,2,932683089,1999-07-22,1999-07-31
4,933875379,2,932683089,1999-07-22,1999-08-05


In [18]:
df.shape

(9601013, 5)

In [19]:
def retention_by_player (reg_data, auth_data, first_cohort, last_cohort, quantity_days):
    """
    reg_data - данные о времени регистрации
    auth_data - данные о времени захода пользователей в игру
    first_cohort - начало периода для анализа, вводится в формате 'гггг-мм-дд'
    last_cohort - окончание периода для анализа, вводится в формате 'гггг-мм-дд'
    quantity_days - количество дней, чтобы посмотреть retertion
    """  
    df = auth_data.merge(reg_data, how='left', on='uid') # объединяем данные
    df['reg_date'] = pd.to_datetime(df.reg_ts, unit='s').dt.to_period('D') # преобразуем данные в дни
    df['auth_date'] = pd.to_datetime(df.auth_ts, unit='s').dt.to_period('D') # преобразуем данные в дни
    cohort = df.groupby(['reg_date', 'auth_date'], as_index=False).agg({'uid': 'nunique'}) # создадим когорты
    cohort['diff_date'] = (cohort.auth_date - cohort.reg_date).apply(attrgetter('n')) # дни от даты регистрации
    cohort_filter = cohort.query('diff_date <= @quantity_days and @last_cohort >= reg_date >= @first_cohort') # фильтр
    cohort_pivot = cohort_filter.pivot_table(index='reg_date', columns='diff_date', values='uid') # развернем
    cohort_null = cohort_pivot.iloc[:,0] # пользователи в нулевой период
    retention = cohort_pivot.div(cohort_null, axis=0) # получаем %
    del retention[0] # удаляем нулевой день для смотрибельности
    
    # преобразуем вид
    ur_style = (retention
                .style 
                .set_caption('Retention by player')           # добавляем подпись 
                .background_gradient(cmap='viridis')          # раскрашиваем ячейки по столбцам 
                .highlight_null('white')                      # делаем белый фон для значений NaN 
                .format("{:.2%}", na_rep=""))                 # числа форматируем как проценты, NaN заменяем на пустоту 
        
    return ur_style

### Пример работы функции retention_by_player

In [20]:
max_cohort = df.reg_date.max()
min_cohort = df.reg_date.min()
print('диапазон дат от',min_cohort, 'до', max_cohort)

диапазон дат от 1998-11-18 до 2020-09-23


In [21]:
# Задаем исходные данные
first_cohort = '2020-09-17'
last_cohort = '2020-09-23'
quantity_days = 7

In [22]:
retention_by_player(reg_data, auth_data, first_cohort, last_cohort, quantity_days)

diff_date,1,2,3,4,5,6
reg_date,,,,,,
2020-09-17,1.66%,4.30%,4.55%,5.47%,6.27%,4.24%
2020-09-18,1.90%,4.54%,4.23%,5.21%,3.74%,
2020-09-19,1.84%,3.55%,4.71%,3.49%,,
2020-09-20,2.44%,4.34%,2.38%,,,
2020-09-21,1.89%,2.99%,,,,
2020-09-22,0.85%,,,,,
2020-09-23,,,,,,
